In [1]:
!pip install sagemaker --upgrade


/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


In [4]:
!pip install "sagemaker>=2.48.0" "transformers==4.6.1" "datasets[s3]==1.6.2" --upgrade


/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 2.2 MB 11.9 MB/s eta 0:00:01
     |████████████████████████████████| 221 kB 52.6 MB/s eta 0:00:01
     |████████████████████████████████| 895 kB 65.9 MB/s eta 0:00:01
     |████████████████████████████████| 3.3 MB 48.7 MB/s eta 0:00:01
     |████████████████████████████████| 747 kB 32.1 MB/s eta 0:00:01
     |████████████████████████████████| 243 kB 61.8 MB/s eta 0:00:01
     |████████████████████████████████| 130 kB 67.1 MB/s eta 0:00:01
     |████████████████████████████████| 7.2 MB 22.3 MB/s eta 0:00:01
     |████████████████████████████████| 

In [2]:
%%capture
import IPython
!conda install -c conda-forge ipywidgets -y
IPython.Application.instance().kernel.do_shutdown(True) # has to restart kernel so changes are used

In [6]:
import sagemaker.huggingface


In [7]:
import sagemaker

sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

role = sagemaker.get_execution_role()
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker role arn: arn:aws:iam::869312567674:role/service-role/AmazonSageMaker-ExecutionRole-20210820T145758
sagemaker bucket: sagemaker-us-east-1-869312567674
sagemaker session region: us-east-1


In [20]:
# load dataset
dataset = load_dataset('imdb', ignore_verifications=True)


Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/4ea52f2e58a08dbc12c2bd52d0d92b30b88c00230b4522801b3636782f625c5b. Subsequent calls will reuse this data.


In [21]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 18345
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 0
    })
})

In [23]:
dataset['train']['text'][0]

'Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as "Teachers". My 35 years in the teaching profession lead me to believe that Bromwell High\'s satire is much closer to reality than is "Teachers". The scramble to survive financially, the insightful students who can see right through their pathetic teachers\' pomp, the pettiness of the whole situation, all remind me of the schools I knew and their students. When I saw the episode in which a student repeatedly tried to burn down the school, I immediately recalled ......... at .......... High. A classic line: INSPECTOR: I\'m here to sack one of your teachers. STUDENT: Welcome to Bromwell High. I expect that many adults of my age think that Bromwell High is far fetched. What a pity that it isn\'t!'

In [28]:

from transformers import AutoTokenizer
# tokenizer used in preprocessing
tokenizer_name = 'distilbert-base-uncased'

# dataset used
dataset_name = 'imdb'

# s3 key prefix for the data
#s3_prefix = 'samples/datasets/imdb'

# download tokenizer
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

# tokenizer helper function
def tokenize(batch):
    return tokenizer(batch['text'], padding='max_length', truncation=True)

# load dataset
train_dataset, test_dataset = load_dataset('imdb', ignore_verifications=True, split=['train', 'test'])
test_dataset = test_dataset.shuffle().select(range(10000)) # smaller the size for test dataset to 10k 


# tokenize dataset
train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
test_dataset = test_dataset.map(tokenize, batched=True, batch_size=len(test_dataset))

# set format for pytorch
#train_dataset =  train_dataset.rename_column("label", "labels")
#train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
#test_dataset = test_dataset.rename_column("label", "labels")
#test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])

Reusing dataset imdb (/root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/4ea52f2e58a08dbc12c2bd52d0d92b30b88c00230b4522801b3636782f625c5b)


In [31]:
train_dataset


Dataset({
    features: ['attention_mask', 'input_ids', 'label', 'text'],
    num_rows: 18345
})

In [33]:
train_dataset['text'][0]

'Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as "Teachers". My 35 years in the teaching profession lead me to believe that Bromwell High\'s satire is much closer to reality than is "Teachers". The scramble to survive financially, the insightful students who can see right through their pathetic teachers\' pomp, the pettiness of the whole situation, all remind me of the schools I knew and their students. When I saw the episode in which a student repeatedly tried to burn down the school, I immediately recalled ......... at .......... High. A classic line: INSPECTOR: I\'m here to sack one of your teachers. STUDENT: Welcome to Bromwell High. I expect that many adults of my age think that Bromwell High is far fetched. What a pity that it isn\'t!'

In [46]:
from sagemaker.huggingface import HuggingFace

# hyperparameters, which are passed into the training job
hyperparameters={'epochs': 1,
                 'train_batch_size': 32,
                 'model_name':'distilbert-base-uncased'
                 }

huggingface_estimator = HuggingFace(entry_point='train.py',
                            source_dir='./',
                            instance_type='ml.p3.2xlarge',
                            instance_count=1,
                            role=role,
                            transformers_version='4.6',
                            pytorch_version='1.7',
                            py_version='py36',
                            hyperparameters = hyperparameters)


In [47]:
s3_prefix = 'huggingface-2021-08-24-15-00-53-465/output'
training_input_path = f's3://{sess.default_bucket()}/{s3_prefix}/train_data'
test_input_path = f's3://{sess.default_bucket()}/{s3_prefix}/test_data'


In [48]:
training_input_path

's3://sagemaker-us-east-1-869312567674/huggingface-2021-08-24-15-00-53-465/output/train_data'

In [ ]:
# starting the train job with our uploaded datasets as input
huggingface_estimator.fit({'train': training_input_path, 'test': test_input_path})

2021-08-24 19:06:34 Starting - Starting the training job...
2021-08-24 19:06:57 Starting - Launching requested ML instancesProfilerReport-1629831561: InProgress
......
2021-08-24 19:07:57 Starting - Preparing the instances for training.........
2021-08-24 19:09:18 Downloading - Downloading input data...
2021-08-24 19:09:58 Training - Downloading the training image..................
2021-08-24 19:13:01 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-08-24 19:13:02,811 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2021-08-24 19:13:02,836 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-08-24 19:13:05,863 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-08-24 19:13:44,137 sagemaker-training-toolkit INFO     Invoking user script

Trai

In [50]:
predictor = huggingface_estimator.deploy(1,"ml.g4dn.xlarge")


--------!

In [51]:
classification_input= {"inputs":"I love using the new device. It was awesome! "}

predictor.predict(classification_input)

[{'label': 'LABEL_1', 'score': 0.9975283145904541}]

In [52]:
classification_input= {"inputs":"I hate using the new device. It was hard to use! "}

predictor.predict(classification_input)

[{'label': 'LABEL_0', 'score': 0.9810707569122314}]

In [53]:
classification_input= {"inputs":"This device is just so so. "}

predictor.predict(classification_input)

[{'label': 'LABEL_0', 'score': 0.7101688385009766}]